Consultar ID de las expansiones para realziar consultas de las cartas de cada expansión

In [1]:
import pandas as pd
import requests

OP_game_id = 15

url =  'https://api.cardtrader.com/api/v2/expansions'

# Tu token de autenticación
auth_token = 'TOKEN_AUTORIZACION_CARDTRADER'

# Cabeceras de la solicitud
headers = {
    "Authorization": f"Bearer {auth_token}"
}

# Parámetros opcionales de la solicitud
params = {
    "game_id": 15 
}

# Realizar la solicitud GET
response = requests.get(url, headers=headers, params=params)

# Verificar el estado de la respuesta
if response.status_code == 200:
    # Imprimir el contenido de la respuesta (en formato JSON)
    response.json()
    df = pd.DataFrame(response.json())
    one_piece_expansions = df[df['game_id'] == OP_game_id]
    
else:
    print(f"Error: {response.status_code} - {response.text}")

Filtrar por expansionas ya lanzadas

In [2]:
ids_op_expan = one_piece_expansions.iloc[:-2,:]
ids_op_expan = ids_op_expan[['id','code']] 

Comsultar nombre y características de las cartas

In [3]:
import json
import time

url = 'https://api.cardtrader.com/api/v2/blueprints/export'

Op_cards = pd.DataFrame()
for id,name in ids_op_expan.values:
    # Parámetros opcionales de la solicitud
    params = {
        "expansion_id": id
    }
    response = requests.get(url, headers=headers, params=params)

    # Verificar el estado de la respuesta
    if response.status_code == 200:
        # Imprimir el contenido de la respuesta (en formato JSON)
        data = response.json()
        df = pd.json_normalize(data)
        Op_cards_exp =  df[['id', 'name', 'expansion_id','fixed_properties.collector_number', 'fixed_properties.onepiece_rarity']]
        Op_cards_exp = Op_cards_exp.dropna()
        Op_cards_exp.columns = ['BlueprintID','Name', 'expansion_id','Code', 'Rarity']
        
        Op_cards = pd.concat([Op_cards, Op_cards_exp], ignore_index=True)
        
    else:
        print(f"Error: {response.status_code} - {response.text}")
    time.sleep(1)

In [6]:
one_piece_expansions.rename(columns={'id': 'expansion_id'}, inplace=True)
cartas_expansiones = pd.merge(Op_cards,one_piece_expansions, how='left', on = 'expansion_id' )

In [7]:
cartas_expansiones = cartas_expansiones.drop(['expansion_id', 'game_id'], axis=1)
cartas_expansiones.columns = ['Card_id','Card_name', 'Card_code','Rarity',	'Expasion_code','Expansion_name']
cartas_expansiones.to_csv('Cartas_One_Piece.csv', index=False)